# Bond Repo Fundamentals

This notebook demonstrates how to access and use the **Bond Repo** functionalities within the **LSEG Financial Analytics SDK**. 

In a Bond Repo, at the start of the transaction (the Purchase Date), one party (the Seller) sells a **Bond** to another party (the Buyer) at one price (the Purchase Price) and commits to repurchase assets which are equivalent to those sold at a future date or on demand (the Repurchase Date) at an agreed or calculable but different price (the Repurchase Price).

**You will be able to:**
- Define Bond Repo with basic parameters
- Configure pricing parameters
- Evaluate Bond Repo Analytics

## Imports


Import the following necessary modules:

- `lseg_analytics.pricing.instruments.repo` - for Bond Repo instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import repo as rp

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Bond Repo instrument you need to follow a structured 4-step process:

1. **Bond Definition** - Specify the underlying Bond parameters 
2. **Repo Definition** - Specify the Repo parameters
3. **Repo Definition Instrument** - Create the instrument object using the Repo Definition
3. **Pricing Preferences** - Configure pricing parameters, optional


In [2]:
# 1.Define an underlying Bond instrument
fixed_bond_definition = rp.BondDefinition(
    notional_ccy = "USD",
    issue_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    end_date = dt.datetime.strptime("2030-01-01", "%Y-%m-%d"),
    fixed_rate_percent = 2,
    interest_payment_frequency = rp.InterestPaymentFrequencyEnum.QUARTERLY,
    interest_calculation_method = rp.InterestCalculationMethodEnum.DCB_ACTUAL_ACTUAL

)

underlying_bond = rp.RepoUnderlyingContract(
        instrument_definition = fixed_bond_definition,
        instrument_type = "Bond"
)
print("Underlying Bond definition created")

# 2.Define Repo instrument
repo_definition = rp.RepoDefinition(
    buy_sell = rp.IPABuySellEnum.BUY,
    start_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    tenor = "1Y",
    underlying_instruments = [underlying_bond]
)

# 3.Create the Repo Instrument from the defintion
repo_instrument = rp.RepoDefinitionInstrument (definition = repo_definition)
print("Repo Instrument definition created")

# 4. Configure pricing parameters, optional
pricing_params = rp.RepoPricingParameters(
    valuation_date = dt.datetime.strptime("2025-07-18", "%Y-%m-%d"),                
)
print("Pricing parameters configured")

Underlying Bond definition created
Repo Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = rp.price(
    definitions = [repo_instrument],
    pricing_preferences = pricing_params
)
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument and Underlying Bond definitions set above. 

 - **pricingPreferences**: pricing parameters set above (e.g valuation date)

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashFlows**: gives details on the different cashflows of the Bond Repo
   - **description**: Instrument summary and also fields not specified with the default values used in the pricing.
   - **pricingAnalysis**: different information on Bond Repo valuation: `haircutRatePercent`, `marketDataDate`, `valuationDatee`, `purchasePrice`, `repurchasePrice`, `underlyingAtStartObject`, `underlyingAtEndObject`
   - **valuation**: `repoInterestAmountInDealCcy`, `reinvestmentIncomeInDealCcy`, `marketValueInDealCcy`
   - **error**: root cause of the issue if any


In [4]:
# Access the description object
description = response.data.analytics[0].description
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentDescription": "Repo 1000000 'USD' 4.37% 2026-01-02 Haircut: 0% / Initial Margin: 1.0000",
    "dealCcy": "USD",
    "startDate": "2025-01-01T00:00:00Z",
    "endDate": "2026-01-02T00:00:00Z",
    "initialMarginPercent": 100.0,
    "nominalPledgeQuantity": 1000000.0,
    "numberOfCoupons": 4,
    "dayCountBasis": "Dcb_Actual_360",
    "repoCurveId": "YIELD_CURVE_USD_RepoCurve_2025-07-18 00:00:00",
    "discountCurveId": "IRCurve_USD-OISSOFR_2025-07-18T00:00:00",
    "settlementConvention": "0WD",
    "repoRatePercent": 4.37113664794044
}


In [5]:
# Access the pricing analysis object
pricing_analysis = response.data.analytics[0]["pricingAnalysis"]
# Convert to DataFrame
df_pricing_analysis = pd.DataFrame(list(pricing_analysis.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_pricing_analysis.head(5))

,Fields,Value
0,haircutRatePercent,0.0
1,marketDataDate,2025-07-18T00:00:00Z
2,valuationDate,2025-07-18T00:00:00Z
3,purchasePrice,1043778.653199
4,repurchasePrice,1069349.750231


In [6]:
# Convert to DataFrame
valuation = response.data.analytics[0].valuation
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_valuation)

,Fields,Value
0,repoInterestAmountInDealCcy,4.575727e+04
1,reinvestmentIncomeInDealCcy,3.344438e+02
2,marketValueInDealCcy,1.053540e+06


## Appendix: Main Functions Reference

The following is a list of main classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.repo`:

**Core Classes:**
- `RepoDefinition` - Define repo instrument specifications
- `BondDefinition` - Define underlying bond instrument specifications
- `RepoUnderlyingContract` - Define the Repo underlying instrument
- `RepoDefinitionInstrument` - Create repo instrument from RepoDefinition
- `RepoPricingParameters` - Configure pricing parameters and market data settings 
- `BondPricingParameters` - Configure pricing parameters and market data settings for the underlying bond
- `RepoUnderlyingPricingParameters` - Configure pricing parameters and market data settings for the underlying instrument

**Response Classes:**
- `RepoCalculationResponse` - Main response container
- `RepoParameters` - Pricing paramenters showed in the pricing response
- `RepoCashflows` - Detailed cash-flows of the Repo
- `RepoDescription` - Instrument summary
- `RepoValuation` - Individual instrument valuation results
- `RepoPricingAnalysis` - Detailed analytics and calculations

**Configuration Enums:**
- `IPABuySellEnum` - Buy/Sell direction (BUY, SELL)
- `RepoRateTypeEnum` - The rate type used to compute repo interest amount (MONEY_MARKET, DISCOUNT, etc.)
- `RepoCurveTypeEnum` - Curve type used to compute repo interest amount (REPO_CURVE (default value), DEPOSIT_CURVE, LIBOR_FIXING)
- `RepoRateFrequencyEnum` - Repo payment frequency  (ANNUAL, SEMI_ANNUAL, QUARTERLY, MONTHLY)
- `DayCountBasisEnum` - The day count basis convention: (DCB_ACTUAL_360 (default value), DCB_30_360, etc.)
- `PriceSideEnum` - Market price sides (BID, ASK, MID)

**Main Functions:**
- `price()` - Execute cap/floor pricing calculation